In [0]:
import pyspark.sql.functions as F
import pandas as pd 

df_btc = spark.table("btc_m2.bronze.btc_raw")
df_liq = spark.table("btc_m2.bronze.liquidity_raw")

pdf_btc = df_btc.select("Date", "Close").toPandas()
pdf_liq = df_liq.select("date", "m2_supply").toPandas()

pdf_btc.columns = ['data', 'preco_btc']
pdf_liq.columns = ['data', 'liquidez_global']

pdf_btc['data'] = pd.to_datetime(pdf_btc['data'])
pdf_liq['data'] = pd.to_datetime(pdf_liq['data'])

df_merged = pd.merge(
    pdf_btc.sort_values('data'), 
    pdf_liq.sort_values('data'), 
    on='data', 
    how='left'
)

df_merged['liquidez_global'] = df_merged['liquidez_global'].interpolate(method='linear')

df_merged['liquidez_global'] = df_merged['liquidez_global'].fillna(method='bfill').fillna(method='ffill')

df_silver = spark.createDataFrame(df_merged)

df_silver.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("btc_m2.silver.consolidado")